### ------------------------------------------------------------------------------------------------------------

In [ ]:
import os

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/impulsecorp/PickStocks.git
    !mv PickStocks/*.py .
    !mv PickStocks/data .
    !pip install -U -qq -r PickStocks/requirements.txt

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
# small hack to prevent Colab error
try:
    from datablock import *
except:
    from datablock import *
from backtesting import Backtest
from backtesting.lib import compute_stats
seed

In [ ]:
%pylab inline

### ------------------------------------------------------------------------------------------------------------

In [ ]:
%%time

data_timeperiod = '15min'
data = get_data('SPY', period=data_timeperiod, nrows=None)
data = procdata_lite(data)

In [ ]:
# for inspectiion
# display(data[0:50])
# print(get_data_features(data))
data.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_class = LogisticRegression

In [ ]:
### We'll try various combinations of all integer features first

In [ ]:
system.train_set_end = 0.4 # percentage point specifying the training set end point (1.0 means all data is training set)
system.val_set_end = 0.7 # percentage point specifying the validation set end point (1.0 means no test set)
# basically this is the data with the values above, which are like sliders determining the layout
# [|0.0| ...... train ........ |0.4| ............ val ............ |0.7| .............. test ............... |1.0|]

clf_class = LogisticRegression

In [ ]:
%%time

# search space boundaries for each feature
space = { 'day' : (0, 5),  # day
          'hour' : (9, 16), # hour
#           'times in row' : (0, 10), # times in row
#           'overnight direction' : (-1, 2), # overnight direction
          }

optkw['method'] = 'skopt'
optkw['max_tries'] = 150


feature_names = sorted(tuple(featformat(x) for x in space.keys()))
space = {featformat(x)+'_target_134':np.arange(*y).tolist() for x,y in sorted(space.items())}
for k,v in space.items():
    setattr(MLSingleMultiParamEqStrategy, k, 0.0)
bt = Backtest(data, MLSingleMultiParamEqStrategy, **btkw)
consts = dict(mode=['opt'], clf_class=[clf_class], period=[data_timeperiod])
results = bt.optimize(**consts, **optkw, **space,
                      random_state=reseed())
plot_optresult(get_optdata(results, consts), 'index');

In [ ]:
# performance of best backtest
plot_result(bt, results)

In [ ]:
# stats for best backtest
display(results[0])

### ------------------------------------------------------------------------------------------------------------

In [ ]:
gr = get_optdata(results, consts)
best_values= gr.index[np.argmax(gr.values)]
best_values, np.max(gr.values)

In [ ]:
ft = {x+'_target_134':y for x,y in zip(feature_names, best_values)}
ft

In [ ]:
%%time

# testing the optimized model on unseen data here
consts['mode'] = ['test']
stats = bt.run(**{x:consts[x][0] for x in consts.keys()}, **ft)
display(stats)

In [ ]:
plot_result(bt, [stats])